# Simple chatbot

## Prerequisites

1. Run this command to create virtual envrionment: `virtualenv langchain --python=python3.11`
2. Activate the environment: `source langchain/bin/activate`
3. Import required packages: `pip install -r requirements.txt`

## Run the app

1. Import package and init base objects: AzureOpenAIEmbeddings and ChromaDB

In [ ]:
import data_loader
import settings
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import Tool
from langchain_chroma import Chroma
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

# Init base objects
azure_embedding = AzureOpenAIEmbeddings(
    model=settings.AZURE_OPENAI_API_EMBEDDING_MODEL,
    azure_endpoint=settings.AZURE_OPENAI_ENDPOINT,
    api_key=settings.AZURE_OPENAI_API_KEY,
    openai_api_version=settings.AZURE_OPENAI_API_VERSION
)

vectordb = Chroma(
    embedding_function=azure_embedding,
    persist_directory="../langchain.db",
    collection_name="langchain"
)

2. Load data from pdf file into ChromaDB

In [ ]:
original_documents = data_loader.load_pdf_file()
enhanced_documents = data_loader.process_documents(original_documents)
vectordb.add_documents(enhanced_documents)

3. Set up agent

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

llm = AzureChatOpenAI(
    azure_deployment=settings.AZURE_OPENAI_API_CHAT_MODEL,
    azure_endpoint=settings.AZURE_OPENAI_ENDPOINT,
    model=settings.AZURE_OPENAI_API_CHAT_MODEL,
    api_version=settings.AZURE_OPENAI_API_VERSION,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    streaming=False
)


@tool
def internal_knowledge_search(query):
    """Search internal knowledge using vector"""
    results = retriever.invoke(query)
    return results


tools = [
            Tool(
                    name="InternalKnownledge",
                    func=internal_knowledge_search,
                    description="Use this tool to search internal knowledge at Nashtech"
                ),
            DuckDuckGoSearchRun(description="Use this tool to search information on the Internet")
        ]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an intelligent assistant with access to multiple tools. \
            Always search InternalKnownledge. \
                Make sure the page_number and source are included in the response"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=5)
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

4. Start the chatbot

In [ ]:
question = ""
while (True):
    question = input("You: ")
    if question == "quit":
        break
    result = agent_executor.invoke({"input": question})
    print(f'AI: {result["output"]}')
    print("====================")